# Tutorial B - route analysis

In this tutorial you will learning the advances analysis techniques for route predictions

After the completion of this tutorial, you will know:
* How to extract routes from retrosynthesis search trees
* How to score routes with AiZynthFinder
* How to score route with rxnutils
* How to calculate route similarities
* How to cluster routes


We will start with installing packages from pypi

In [ ]:
!pip install --quiet aizynthfinder
!pip install --quiet reaction-utils[models]
!pip install --ignore-installed Pillow==9.0.0

### Setup

As with the basic tutorial we will work with public data and models

In [ ]:
!mkdir --parents data && download_public_data data

And we will setup the `aizynthfinder` interface similarly to the basic tutorial as well...


In [ ]:
import logging
from aizynthfinder.utils.logging import setup_logger
setup_logger(logging.INFO)

from aizynthfinder.aizynthfinder import AiZynthFinder

In [ ]:
finder = AiZynthFinder("data/config.yml")
finder.stock.select_all()
finder.expansion_policy.select("uspto")

... and run retrosynthesis analysis on amenamevir

In [ ]:
finder.target_smiles = "Cc1cccc(C)c1N(CC(=O)Nc1ccc(-c2ncon2)cc1)C(=O)C1CCS(=O)(=O)CC1"
display(finder.target_mol.rd_mol)
finder.tree_search(show_progress=True)

### Extracting routes

In the previous tutorial we used the `build_routes` method to extract routes from the search. By default this only returns a limited set of routes based on the score used to guide the search.

In [ ]:
finder.build_routes()
len(finder.routes)

We can control how many routes that we want to generate. The algorithm is works on a minimum and maximum number of routes, but more than the minimum can be returned if multiple routes have the same score but never more than the maximum

In [ ]:
from aizynthfinder.analysis.utils import RouteSelectionArguments
sel_args = RouteSelectionArguments(nmin=10, nmax=15)
finder.build_routes(sel_args)
len(finder.routes)

In [ ]:
sel_args = RouteSelectionArguments(nmin=10)
finder.build_routes(sel_args)
len(finder.routes)

We can also make it return all solved routes, i.e. routes leading to starting material in stock.

In [ ]:
sel_args = RouteSelectionArguments(return_all=True)
finder.build_routes(sel_args)
len(finder.routes)

### Scoring

Now we will explore some ways to score the generated routes. We have a preview of this in the previous tutorial and now we will dig into this some more.

There are a number of route scores available in the `aizynthfinder` package but we will also use some more ellaborate route scores available in the `rxnutils` package.

In `aizynthfinder`, scores available are typically accessible through `finder.scorers`, which is a collection of scorer objects. These can be loaded from the configuration file or created by users.

There are a number of scores that are loaded automatically, because they are simple, fast, and does not require any external input.

In [ ]:
finder.scorers.names()

The scorers are:

- "state score", the score used to guide the tree search (a combination of the number of steps in the route and the fraction of starting material in stock)
- "number of reactions", the total number of steps in the route
- "number of pre-cursors", the number of starting materials
- "number of pre-cursors in stock", the number of starting materials in stock
- "average template occurrence", the average database occurence of the templates used in the route

We can then use them to score the routes...

In [ ]:
finder.build_routes()
finder.routes.compute_scores(*finder.scorers.objects())
finder.routes.all_scores[0]


... and we can put them in a nice table using Pandas

In [ ]:
import pandas as pd
pd.DataFrame(
    finder.routes.all_scores
)

Next, we can instantiate another available scorer that is a route score suggested by [Badowski and co-workers](https://pubs.rsc.org/en/content/articlelanding/2019/sc/c8sc05611k).

The limitation with this score for our current experiment is that:
- we don't have a stock that provides cost of the starting material
- we don't have a cost of each reaction
- we don't have an estimate of the yield in each step

Therefore, we are using an assumed yield of 0.8, a reaction cost of 1 and the cost of starting material is 1 for anything that is in stock and 10 for everything else.

In [ ]:
from aizynthfinder.context.scoring import RouteCostScorer
scorer = RouteCostScorer(finder.config)
finder.routes.compute_scores(scorer)
pd.DataFrame(
    finder.routes.all_scores
)

Finally, we will create a custom scorer that takes the difference compared to the maximum depth of the search and scales it with a sigmoid-like function

To implement a custom scorer, we need to implement at least three functions
- `__repr__` that returns a string label for the scorer. This is what is shown as the column headers above.
- `_score_node` that should implement the scoring for the nodes in the search tree
- `_score_reaction_tree` that should implement the scoring for a synthesis route

(here we are cheating a bit on the `_score_reaction_tree` function, it is not exactly like the `_score_node`. See if you can spot the difference!)

In [ ]:
from aizynthfinder.context.scoring.scorers import Scorer

class DeltaNumberOfTransformsScorer(Scorer):

    def __init__(self, config):
      super().__init__(
          config,
          scaler_params={"name": "squash", "slope": -1, "yoffset": 0, "xoffset": 3},
      )

    def __repr__(self):
        return "delta number of transforms"

    def _score_node(self, node):
        return self._config.search.max_transforms - node.state.max_transforms

    def _score_reaction_tree(self, tree):
        return self._config.max_transforms - len(list(tree.reactions()))

custom_scorer = DeltaNumberOfTransformsScorer(finder.config)
finder.routes.compute_scores(custom_scorer)
pd.DataFrame(
    finder.routes.all_scores
)

### Scoring with `rxnutils`

Now, we will have a look at some route scoring algorithms that are available in the `rxnutils`package.

Before we do this, we will assign classification to the predicted reactions based on the template hash. This is because the scores we will use are based on the the NextMove reaction class, which is missing from the public USPTO model

In [ ]:
hash2class = {
    '00cfc3ff089c8152916b2b19d242a80c85c8d70fda9ab8b0e8a1e7b52bf3380d': '2.1.1',
    '110b771d77121b19f0a012d7c67263575768c8985321b84bf1629ea79c879d51': '6.2.2',
    '19f2a0f4c6a46a956748171b40376b6f742843423bcde2abbd8b3a6cf6b0a7b9': '9.3.1',
    '21e32ba7f5a8b5ab7b02ba748a13ecbd720a7475e580f0e1ebc3ce935e54b5a2': '2.1.1',
    '264a7579c52f48f3c97db6b8bc61252d596b1a7b85cd90bbc48eca4a4c46134c': '2.1.1',
    '3d2ded365fb5f402223c094af5dc49e39510bb1325a199cb96a43a0828c1a888': '2.1.2',
    '49f31ca31dd097e43d87899a3976cdfcec8d77ac8f69c7b16a07a0d183b5f742': '1.3.6',
    '634b5eedc2112400cedf8f3edde0c3e221ba4f01b8b7f27953b2811c42fc31f1': '2.1.10',
    '6d0c1070464abadf312613f8d200b60386b42b62476555a9b48aad53e2e82868': '6.2.1',
    '866e1a8ac889f536ad4f364036065c6e47f9d15c5f533dbb707d5e47a3434b1a': '9.3.1',
    '8bdb325ee96ea0b293bc31507a025e93f4907432629dbb7ba3f79c7ed22c5afe': '3.1.2',
    'b12cae7f7da913000dcdba7def39559511339ce3330a7426c5ca0d488d1da652': '2.1.1',
    'c42f96a0658c03bc12dc20efce06e66daaeccc675d73631eea24ea610a7227b8': '7.9.2',
    'e913b1094150d416bf8203e6e8c96f70bb606ea82d757cc0bdffcc4e7fad5f5a': '2.6.3'
}
for reaction_tree in finder.routes.reaction_trees:
    for reaction in reaction_tree.reactions():
        template_hash = reaction.metadata["template_hash"]
        reaction.metadata["classification"] = hash2class.get(template_hash, reaction.metadata["classification"])

Then we will convert our `aizynthfinder` routes to the internal `SynthesisRoute` object of `rxnutils`.

In [ ]:
from rxnutils.routes.readers import read_aizynthfinder_dict
routes = [
    read_aizynthfinder_dict(reaction_tree.to_dict())
    for reaction_tree in finder.routes.reaction_trees]
routes[0]

These objects have many similar functionality to routes in `AiZynthFinder`. Like you can extract metadata or analyze the structure of the route

In [ ]:
routes[0].reaction_data()

In [ ]:
routes[0].leaves()

Next, we will download some data and model files that we will need to score our routes.

In [ ]:
!mkdir files_scoring
!wget https://zenodo.org/records/14533779/files/reaction_class_ranks.csv?download=1 -O files_scoring/reaction_class_ranks.csv
!wget https://zenodo.org/records/14533779/files/deepset_route_scoring_sdf.onnx?download=1 -O files_scoring/deepset_route_scoring_sdf.onnx
!wget https://zenodo.org/records/14533779/files/scscore_model_1024_bits.onnx?download=1 -O files_scoring/scscore_model_1024_bits.onnx

We will start with a score based on a ranking of the reaction classes. We have analysed the internal reaction data of AstraZeneca and has classified NextMove reaction classes based on how often such reactions have been carried out and how often tail succeed.

We will also favour the most highly ranked classes. This could also be used to favour some reaction classes that are ranked low when looking at historial data but is preferred for some other reason.

In [ ]:
df = pd.read_csv("files_scoring/reaction_class_ranks.csv", sep = ",")
reaction_class_ranks = dict(zip(df["reaction_class"], df["rank_score"]))
preferred_classes = df[df["rank_score"]>=5].reaction_class

In [ ]:
from rxnutils.routes.scoring import reaction_class_rank_score
scores_df = pd.DataFrame(finder.routes.all_scores)
scores_df["reaction class rank"] = [
    reaction_class_rank_score(route, reaction_class_ranks, preferred_classes)
    for route in routes
]
scores_df

Next, we will use a trained model developed by [Kaski and co-workers](https://chemrxiv.org/engage/chemrxiv/article-details/67acb4f06dde43c90896605c).

This requires both a trained SCScore model for featurizing the reactions, and the deepset model itself.

In [ ]:
from rxnutils.chem.features.sc_score import SCScore
from rxnutils.routes.deepset.scoring import DeepsetModelClient, deepset_route_score

# Setup SCScore model
scscorer = SCScore("files_scoring/scscore_model_1024_bits.onnx")
# Setup the Deepset model client
deepset_client = DeepsetModelClient("files_scoring/deepset_route_scoring_sdf.onnx")

we will then compute the raw learned score, which is an abstract distance to the experimental routes used for training the model. The lower score the better.

but we will also correct this for route length, i.e. the number of steps, based on expert evaluation of predicted routes.

In [ ]:
scores_df["deepset score"] = [
    deepset_route_score(route, deepset_client, scscorer, reaction_class_ranks)
    for route in routes
]
scores_df["expert augment score"] =  0.97 * scores_df["deepset score"] -0.43 * scores_df["number of reactions"]
scores_df

### Route similarity and clustering

In `rxnutils` there are routines to compute the similarity of routes based on an algorithm from [Genheden and Shields](https://pubs.rsc.org/en/content/articlelanding/2025/dd/d4dd00292j).

We can visualize these similarities using a heatmap


In [ ]:
sel_args = RouteSelectionArguments(nmin=10)
finder.build_routes(sel_args)
routes_large_set = [
    read_aizynthfinder_dict(reaction_tree.to_dict())
    for reaction_tree in finder.routes.reaction_trees
]

In [ ]:
import seaborn as sn
from rxnutils.routes.comparison import simple_route_similarity
similarities = simple_route_similarity(routes)
sn.heatmap(similarities)

we see that route 1 is rather different from most other routes.

You can visualize routes 0 and 1 and try to figure out why they are different.

In [ ]:
routes[0].image()

In [ ]:
routes[1].image()

Next, we will show how you can cluster routes. But we will start plotting the dendrogram.  

In [ ]:
import numpy as np
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering

model2 = AgglomerativeClustering(linkage="single", metric="precomputed", n_clusters=None, distance_threshold=0.0)
model2.fit(1.0-similarities)
counts = np.zeros(len(model2.distances_))
matrix = np.column_stack([model2.children_, model2.distances_, counts])
_ = dendrogram(
    matrix,
    color_threshold=0.0,
    labels=np.arange(0, len(routes)),
)

This is of course reflecting what we saw in the heatmap. That route 1 is rather disimilar to the othe routes, and that route 0 is most similar to routes 4-6.

If try to create 3 clusters from these routes, we find that most routes end up in a single big cluster

In [ ]:
model = AgglomerativeClustering(linkage="single", metric="precomputed", n_clusters=3)
model.fit(1.0-similarities)
model.labels_

Try to experiment with extracting more routes from the search and repeat the analysis of scores, similarities and clusters.

In the next tutorial, we will explore how to adjust the retrosynthesis search in some various ways.